In [2]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import os
import pandas as pd
import requests
from bs4 import BeautifulSoup
import io
import random
import time
from requests.exceptions import ConnectionError, Timeout, RequestException

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
#Load the data
file_path = '/Users/peter/Public/RA/sec_links_remainingtext.csv' ## change this to the path of the file

def try_read_csv(file_path):
    encodings = ['utf-8', 'ISO-8859-1', 'cp1252']  # We can add more encodings if needed
    for i in encodings:
        try:
            return pd.read_csv(file_path, encoding=i)
        except UnicodeDecodeError as e:
            print(f"Failed with encoding {i}: {e}")
        except Exception as e:
            print(f"An error occurred: {e}")
    raise ValueError("Could not read the file with any of the provided encodings.")

data = try_read_csv(file_path)

In [13]:
# Display the first few rows of the dataset
# URL to scrape (you can replace this with a valid URL)
url = "https://www.sec.gov/litigation/admin/ia-1984"

# List of User-Agents to rotate
user_agents = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0',
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
]

# 
def fetch_html_content(url, retries=3):
    session = requests.Session()
    headers = {
        'User-Agent': random.choice(user_agents),
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
        'Accept-Language': 'en-US,en;q=0.5',
        'Referer': 'https://www.google.com/',
        'Connection': 'keep-alive',
        'Upgrade-Insecure-Requests': '1',
        'DNT': '1',  # Do Not Track Request Header
    }
    for attempt in range(retries):
        try:
            response = session.get(url, headers=headers, timeout=10)
            response.raise_for_status()  # Raise HTTPError for bad responses (4xx and 5xx)
            soup = BeautifulSoup(response.content, 'html.parser')
            
            # Target specific content inside the main section of the page
            main_content = soup.find('div', {'id': 'content'})
            if main_content:
                text = main_content.get_text(separator='\n', strip=True)
            else:
                text = soup.get_text(separator='\n', strip=True)
            
            # Random delay to avoid hitting rate limits
            time.sleep(random.uniform(5, 10))
            return text
        except (ConnectionError, Timeout, RequestException) as e:
            print(f"Attempt {attempt + 1} failed with error: {e}. Retrying in {2 ** attempt} seconds...")
            time.sleep(2 ** attempt)  # Exponential backoff
    return None  # Return None if all attempts fail

# Fetch content from the URL
content = fetch_html_content(url)

# Check if content was successfully fetched
if content:
    print("Successfully fetched and cleaned content:")
    
    with open("sec_fetched_content.txt", "w", encoding="utf-8") as file:
        file.write(content)
    print("Content saved to sec_fetched_content.txt")
else:
    print("Failed to fetch content after multiple attempts.")


Successfully fetched and cleaned content:
Content saved to sec_fetched_content.txt


In [5]:
def fetch_txt_content(url, retries=3):
    session = requests.Session()
    headers = {
        'User-Agent': random.choice(user_agents),
        'Accept': 'text/plain,*/*;q=0.8',
        'Accept-Language': 'en-US,en;q=0.5',
        'Referer': 'https://www.google.com/',
        'Connection': 'keep-alive',
        'DNT': '1',  # Do Not Track Request Header
    }
    for attempt in range(retries):
        try:
            response = session.get(url, headers=headers, timeout=10)
            response.raise_for_status()  # Raise HTTPError for bad responses (4xx and 5xx)
            
            text = response.text
            
            # Random delay to avoid hitting rate limits
            time.sleep(random.uniform(5, 10))
            return text
        except (ConnectionError, Timeout, RequestException) as e:
            print(f"Attempt {attempt + 1} failed with error: {e}. Retrying in {2 ** attempt} seconds...")
            time.sleep(2 ** attempt)  # Exponential backoff
    return None  # Return None if all attempts fail

txt_url = "https://www.sec.gov/files/litigation/admin/3-8967.txt"

txt_content = fetch_txt_content(txt_url)
if txt_content:
    print("Successfully fetched TXT content.")
else:
    print("Failed to fetch TXT content after multiple attempts.")

Successfully fetched TXT content.


In [6]:
txt_content

'\r\n\r\n\r\n\r\n\r\n\r\n                     UNITED STATES OF AMERICA\r\n                            Before the\r\n                SECURITIES AND EXCHANGE COMMISSION\r\n\r\nSECURITIES EXCHANGE ACT OF 1934\r\nRelease No. 36921 / March 5, 1996                    \r\n\r\nACCOUNTING AND AUDITING ENFORCEMENT\r\nRelease No.  767  / March 5, 1996\r\n\r\nADMINISTRATIVE PROCEEDING\r\nFile No. 3-8967            \r\n\r\n___________________________________\r\n                                   :\r\nIn the Matter of                   :    ORDER INSTITUTING\r\n                                   :    PROCEEDINGS AND OPINION\r\nJOHN M. GOLDBERGER, CPA, and       :    AND ORDER PURSUANT TO\r\n                                   :    RULE 102(e) OF THE\r\nC. KIRK FRENCH, CPA,               :    COMMISSION\'S RULES OF\r\n                                   :    PRACTICE\r\n               Respondents.        :\r\n___________________________________:\r\n\r\n                                I.\r\n\r\n     The

In [19]:
data_html = data[data['link_type'] == 'html']
data_txt = data[data['link_type'] == 'txt']



In [20]:
# Fetch content from html first and update DataFrame
for index, row in data_html.iterrows():
    content = fetch_html_content(row['respondents_url'])
    if content:
        data_html.at[index, 'text'] = content
    else:
        data_html.at[index, 'text'] = 'Failed to fetch content'

/var/folders/vs/hhhp3tjj11b7n4545rg5s7bh0000gn/T/ipykernel_94434/4207989831.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_html.at[index, 'text'] = content
/var/folders/vs/hhhp3tjj11b7n4545rg5s7bh0000gn/T/ipykernel_94434/4207989831.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'INVESTMENT ADVISERS ACT OF 1940
RELEASE NO.   1984 / September 28, 2001
INVESTMENT COMPANY ACT OF 1940
RELEASE NO. 25200 / September 28, 2001
ADMINISTRATIVE PROCEEDING
File No. 3-10606
In the Matter of
DUFF & PHELPS INVESTMENT
MANAGEMENT CO., INC.,
Respondent.
:
:
:
:
:
:
:
ORDER INSTITUTING PROCEEDINGS
PURSUANT TO SECTIONS 203(e)
AND 203(k) OF THE INVESTMENT ADVISERS
ACT OF 1940 AND SEC

Attempt 1 failed with error: 503 Server Error: Service Unavailable for url: https://www.sec.gov/litigation/admin/34-43372. Retrying in 1 seconds...
Attempt 2 failed with error: 503 Server Error: Service Unavailable for url: https://www.sec.gov/litigation/admin/34-43372. Retrying in 2 seconds...
Attempt 3 failed with error: 503 Server Error: Service Unavailable for url: https://www.sec.gov/litigation/admin/34-43372. Retrying in 4 seconds...
Attempt 1 failed with error: 503 Server Error: Service Unavailable for url: https://www.sec.gov/litigation/admin/34-43373. Retrying in 1 seconds...
Attempt 2 failed with error: 503 Server Error: Service Unavailable for url: https://www.sec.gov/litigation/admin/34-43373. Retrying in 2 seconds...
Attempt 3 failed with error: 503 Server Error: Service Unavailable for url: https://www.sec.gov/litigation/admin/34-43373. Retrying in 4 seconds...
Attempt 1 failed with error: 503 Server Error: Service Unavailable for url: https://www.sec.gov/litigation/admin

In [24]:
data_html

,release_no,respondents_url,link_type,text
2,3-10606,https://www.sec.gov/litigation/admin/ia-1984,html,INVESTMENT ADVISERS ACT OF 1940\nRELEASE NO. ...
3,3-10607,https://www.sec.gov/litigation/admin/ia-1985,html,INVESTMENT ADVISERS ACT OF 1940\nRelease No. ...
485,33-10857,https://www.sec.gov/litigation/admin/33-10857,html,Failed to fetch content
542,33-10979,https://www.sec.gov/litigation/admin/33-10979,html,Failed to fetch content
655,33-11184,https://www.sec.gov/litigation/admin/33-11184,html,Failed to fetch content
...,...,...,...,...
22560,IC-26752,https://www.sec.gov/litigation/admin/33-8534,html,Securities Act of 1933\nRelease No. 8534 / Feb...
22561,IC-26753,https://www.sec.gov/litigation/admin/ia-2352,html,INVESTMENT ADVISERS ACT OF 1940\nRelease No. ...
22562,IC-26755,https://www.sec.gov/litigation/admin/ia-2354,html,INVESTMENT ADVISERS ACT OF 1940\nRelease No. ...
22563,IC-26756,https://www.sec.gov/litigation/admin/33-8538,html,Securities Act of 1933\nRelease No. 8538 / Feb...


In [23]:
csv_path_html = 'sec_releases_html_with_text.csv'
data_html.to_csv(csv_path_html, index=False)

In [25]:
data_html.to_excel('sec_releases_html_with_text.xlsx', index=False)

ModuleNotFoundError: No module named 'openpyxl'

In [26]:
data_html2 = data_html.copy()

In [34]:
data_html2

,release_no,respondents_url,link_type,text
2,3-10606,https://www.sec.gov/litigation/admin/ia-1984,html,INVESTMENT ADVISERS ACT OF 1940\nRELEASE NO. ...
3,3-10607,https://www.sec.gov/litigation/admin/ia-1985,html,INVESTMENT ADVISERS ACT OF 1940\nRelease No. ...
485,33-10857,https://www.sec.gov/litigation/admin/33-10857,html,Failed to fetch content
542,33-10979,https://www.sec.gov/litigation/admin/33-10979,html,Failed to fetch content
655,33-11184,https://www.sec.gov/litigation/admin/33-11184,html,Failed to fetch content
...,...,...,...,...
22560,IC-26752,https://www.sec.gov/litigation/admin/33-8534,html,Securities Act of 1933\nRelease No. 8534 / Feb...
22561,IC-26753,https://www.sec.gov/litigation/admin/ia-2352,html,INVESTMENT ADVISERS ACT OF 1940\nRelease No. ...
22562,IC-26755,https://www.sec.gov/litigation/admin/ia-2354,html,INVESTMENT ADVISERS ACT OF 1940\nRelease No. ...
22563,IC-26756,https://www.sec.gov/litigation/admin/33-8538,html,Securities Act of 1933\nRelease No. 8538 / Feb...


  Using cached openpyxl-3.1.2-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached et_xmlfile-1.1.0-py3-none-any.whl.metadata (1.8 kB)
Using cached openpyxl-3.1.2-py2.py3-none-any.whl (249 kB)
Using cached et_xmlfile-1.1.0-py3-none-any.whl (4.7 kB)
Note: you may need to restart the kernel to use updated packages.


In [43]:
data_html2 = pd.DataFrame(data_html2)

file_name = 'sec_html_data.xlsx'
data_html2.to_excel(file_name)